# SQL Outline

## Day 1

* AWS Overview
  * EC2, RDS

* Tasks
  * log into AWS Console
  * create EC2 instance
    * yum install mysql
    * wget .sql
    
## Day 2

* RDS
* MySQL

* Tasks
  * create RDS instance
  * load SQL file from EC2
  * browse with MySQLWorkbench
  
## Day 3

* pymysql

* Tasks
  * query goodreads
  
## Day 4

* BigQuery, Homework


## Introduction to pymysql



In [3]:
import pymysql

In [1]:
rds_endpoint = ""
username = ""
pw = ""
database=''

In [5]:
connection = pymysql.connect(host=rds_endpoint, # remove this before submitting
                             user=username, # remove this before submitting
                             password=pw, # remove this before submitting
                             db=database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [6]:
with connection.cursor() as cursor:
    cursor.execute('select * from books where original_title like "%Hunger Games%"')
    results = cursor.fetchall()
    for result in results:
        print(result['original_title'])

The Hunger Games
The Hunger Games Box Set


### <font color="magenta">Q1: Find the book titles that contain 'narnia' in the name.

In [12]:
with connection.cursor() as cursor:
    cursor.execute('select * from books where original_title like "%narnia%"')
    results = cursor.fetchall()
    for result in results:
        print(result['original_title'])

The Chronicles of Narnia
Prince Caspian: The Return to Narnia
The Chronicles of Narnia: The Lion, the Witch and The Wardrobe (Sheet Music)
The Chronicles of Narnia - The Lion, the Witch, and the Wardrobe Official Illustrated Movie Companion


### COUNT

In [7]:
with connection.cursor() as cursor:
    cursor.execute('select count(*) from books where original_title like "%Hunger Games%"')
    result = cursor.fetchone()
    print(result)

{'count(*)': 2}


### DISTINCT

In [51]:
with connection.cursor() as cursor:
    cursor.execute('select DISTINCT authors from books')
    results = cursor.fetchall()
    print(len(results))

4669


### <font color="magenta">Q2: What languages are represented in this dataset of books?

In [13]:
with connection.cursor() as cursor:
    cursor.execute('select DISTINCT language_code from books')
    results = cursor.fetchall()
    for result in results:
        print(result)

{'language_code': 'eng'}
{'language_code': 'en-US'}
{'language_code': 'en-CA'}
{'language_code': ''}
{'language_code': 'spa'}
{'language_code': 'en-GB'}
{'language_code': 'fre'}
{'language_code': 'nl'}
{'language_code': 'ara'}
{'language_code': 'por'}
{'language_code': 'ger'}
{'language_code': 'nor'}
{'language_code': 'jpn'}
{'language_code': 'en'}
{'language_code': 'vie'}
{'language_code': 'ind'}
{'language_code': 'pol'}
{'language_code': 'tur'}
{'language_code': 'dan'}
{'language_code': 'fil'}
{'language_code': 'ita'}
{'language_code': 'per'}
{'language_code': 'swe'}
{'language_code': 'rum'}
{'language_code': 'mul'}
{'language_code': 'rus'}


In [52]:
with connection.cursor() as cursor:
    cursor.execute('select COUNT(DISTINCT authors) from books')
    result = cursor.fetchone()
    print(result)

{'COUNT(DISTINCT authors)': 4669}


## SQL from pandas

In [15]:
import pandas as pd

In [16]:
df = pd.read_sql('select * from books',con=connection)
len(df)

10000

In [17]:
df = pd.read_sql('select * from books where original_title = "The Hunger Games"',con=connection)
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


### <font color="magenta"> Q3: In what year was Suzanne Collins' first book published?

In [20]:
df = pd.read_sql('select * from books where authors = "Suzanne Collins"',con=connection)
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,17,6148028,6148028,6171458,201,439023491,9.780440e+12,Suzanne Collins,2009,Catching Fire,...,1831039,1988079,88538,10492,48030,262010,687238,980309,https://images.gr-assets.com/books/1358273780m...,https://images.gr-assets.com/books/1358273780s...
2,20,7260188,7260188,8812783,239,439023513,9.780440e+12,Suzanne Collins,2010,Mockingjay,...,1719760,1870748,96274,30144,110498,373060,618271,738775,https://images.gr-assets.com/books/1358275419m...,https://images.gr-assets.com/books/1358275419s...
3,507,7938275,7938275,11349083,78,545265355,9.780550e+12,Suzanne Collins,2010,The Hunger Games Box Set,...,129665,159760,6429,1015,2521,13438,42932,99854,https://images.gr-assets.com/books/1360094673m...,https://images.gr-assets.com/books/1360094673s...
4,1531,262430,262430,524491,75,043977134X,9.780440e+12,Suzanne Collins,2003,Gregor the Overlander,...,66823,72486,5376,1705,3653,14937,25516,26675,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
5,2935,537070,537070,2432390,45,043979143X,9.780440e+12,Suzanne Collins,2007,Gregor and the Code of Claw,...,33286,35387,1624,645,990,4751,11340,17661,https://images.gr-assets.com/books/1365854331m...,https://images.gr-assets.com/books/1365854331s...
6,3179,385706,385706,860418,53,439656249,9.780440e+12,Suzanne Collins,2005,Gregor and the Curse of the Warmbloods,...,31854,34501,1280,375,813,5632,12527,15154,https://images.gr-assets.com/books/1397852674m...,https://images.gr-assets.com/books/1397852674s...
7,3712,385742,385742,860408,59,439650763,9.780440e+12,Suzanne Collins,2004,Gregor and the Prophecy of Bane,...,29586,32127,1683,235,691,5583,12541,13077,https://images.gr-assets.com/books/1337457481m...,https://images.gr-assets.com/books/1337457481s...
8,4720,319644,319644,7792859,49,439791464,9.780440e+12,Suzanne Collins,2006,Gregor and the Marks of Secret,...,23727,25671,1041,178,557,4117,9617,11202,https://images.gr-assets.com/books/1397854344m...,https://images.gr-assets.com/books/1397854344s...


In [56]:
df = pd.read_sql('select * from books where original_title like "%Hunger Games%"',con=connection)
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,507,7938275,7938275,11349083,78,545265355,9.780550e+12,Suzanne Collins,2010,The Hunger Games Box Set,...,129665,159760,6429,1015,2521,13438,42932,99854,https://images.gr-assets.com/books/1360094673m...,https://images.gr-assets.com/books/1360094673s...


In [57]:
df = pd.read_sql('select * from books where original_title REGEXP ".*Hunger Games.*"',con=connection)
df

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,507,7938275,7938275,11349083,78,545265355,9.780550e+12,Suzanne Collins,2010,The Hunger Games Box Set,...,129665,159760,6429,1015,2521,13438,42932,99854,https://images.gr-assets.com/books/1360094673m...,https://images.gr-assets.com/books/1360094673s...


In [58]:
df = pd.read_sql('select * from books where original_publication_year in (2008, 2010)',con=connection)
df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,20,7260188,7260188,8812783,239,439023513,9.780440e+12,Suzanne Collins,2010,Mockingjay,...,1719760,1870748,96274,30144,110498,373060,618271,738775,https://images.gr-assets.com/books/1358275419m...,https://images.gr-assets.com/books/1358275419s...
2,56,1162543,1162543,2960529,183,031606792X,9.780320e+12,Stephenie Meyer,2008,Breaking Dawn,...,1070245,1107709,44550,100994,114893,213402,267706,410714,https://images.gr-assets.com/books/1361039438m...,https://images.gr-assets.com/books/1361039438s...
3,73,1656001,1656001,3328799,161,316068047,9.780320e+12,Stephenie Meyer,2008,The Host,...,749780,777560,39778,44215,62501,154906,227180,288758,https://images.gr-assets.com/books/1318009171m...,https://images.gr-assets.com/books/1318009171s...
4,88,6442769,6442769,3364505,178,014241493X,9.780140e+12,John Green,2008,Paper Towns,...,461311,655271,42717,14105,47183,154479,223895,215609,https://images.gr-assets.com/books/1349013610m...,https://images.gr-assets.com/books/1349013610s...


In [81]:
df = pd.read_sql("""
                SELECT books.original_title, ratings.rating 
                FROM books,ratings 
                WHERE books.authors LIKE '%Rowling%' 
                AND books.book_id = ratings.book_id;
                """, con = connection)   

In [82]:
df.head()

,original_title,rating
0,Harry Potter and the Philosopher's Stone,5
1,Harry Potter and the Philosopher's Stone,4
2,Harry Potter and the Philosopher's Stone,4
3,Harry Potter and the Philosopher's Stone,5
4,Harry Potter and the Philosopher's Stone,3


In [62]:
df.groupby(by='original_title')[['rating']].mean()

,rating
original_title,
,4.588123
Career of Evil,4.134006
Complete Harry Potter Boxed Set,4.659530
Fantastic Beasts and Where to Find Them,3.775901
Fantastic Beasts and Where to Find Them: The Original Screenplay,4.384259
Harry Potter Boxed Set Books 1-4,4.605357
"Harry Potter Collection (Harry Potter, #1-6)",4.699571
Harry Potter and the Chamber of Secrets,4.229418
"Harry Potter and the Cursed Child, Parts One and Two",3.694737


In [64]:
df = pd.read_sql("select books.original_title, ratings.rating from books,ratings where books.authors like '%Rowling%' and books.book_id = ratings.book_id;", con = connection)
df.head()

,original_title,rating
0,Harry Potter and the Philosopher's Stone,5
1,Harry Potter and the Philosopher's Stone,4
2,Harry Potter and the Philosopher's Stone,4
3,Harry Potter and the Philosopher's Stone,5
4,Harry Potter and the Philosopher's Stone,3


In [70]:
df = pd.read_sql("""
                 SELECT books.original_title, ratings.rating 
                 FROM books,ratings 
                 WHERE books.authors LIKE '%Rowling%' 
                 AND books.book_id = ratings.book_id;
                 """, con = connection)
df.head()

,original_title,rating
0,Harry Potter and the Philosopher's Stone,5
1,Harry Potter and the Philosopher's Stone,4
2,Harry Potter and the Philosopher's Stone,4
3,Harry Potter and the Philosopher's Stone,5
4,Harry Potter and the Philosopher's Stone,3


In [71]:
df.shape

(136631, 2)

In [79]:
df = pd.read_sql("""
                 SELECT books.original_title, ratings.rating 
                 FROM books LEFT JOIN ratings ON books.book_id = ratings.book_id
                 WHERE books.authors LIKE '%Rowling%';
                 """, con = connection)
df.head()

,original_title,rating
0,Harry Potter and the Philosopher's Stone,5
1,Harry Potter and the Philosopher's Stone,4
2,Harry Potter and the Philosopher's Stone,4
3,Harry Potter and the Philosopher's Stone,5
4,Harry Potter and the Philosopher's Stone,3


In [77]:
df.shape

(136631, 2)

# END OF NOTEBOOK: REMEMBER TO STRIP IDETIFYING INFORMATION FROM TOP OF NOTEBOOK!
Please submit both ipynb and html formats of this notebook to Canvas.